In [ ]:
import bottle
import os
from os import path
import json
from hdbcli import dbapi
from hana_ml import dataframe

schema = os.environ.get("validation-schema")
json_object = json.loads(schema)

hana_host = json_object["host"]
hana_port = json_object["port"]
hana_schema = json_object["schema"]
hana_user = json_object["user"]
hana_password = json_object["password"]

def main(event, context):
  if event["extensions"]["request"].path == "/validatefields":
    cc = dataframe.ConnectionContext(hana_host, hana_port, hana_user, hana_password)
    results= {}
    payload = event['extensions']['request'].json

    ioiId = payload.get('ioiId')
    campaignId = payload.get('campaignId')
    country = payload.get('country')
    countryDesc = payload.get('countryDesc')
    mastercode = payload.get('mastercode')

    if ioiId:
      query = f"SELECT ITEM_OF_INTEREST_NAME FROM VALIDATION.IA_ITEM_OF_INTEREST WHERE ITEM_OF_INTEREST_ID = '{ioiId}' AND ITEM_OF_INTEREST_NAME NOT LIKE '%Z(OBSOLETE)%'" 
      print("ioiid" + str(cc.sql(query).collect().empty))
      if cc.sql(query).collect().empty:
        results['ioiID']= 'INVALID'
      else:
        results['ioiID']= 'VALID'
    
    if campaignId:
      query = f"SELECT CAMPAIGN_ID FROM VALIDATION.CAMPAIGNDATA WHERE CAMPAIGN_ID = '{campaignId}' AND ACTIVE_STATUS = True"
      #print("campaign" + str(cc.sql(query).collect().empty))
      if cc.sql(query).collect().empty:
        results['campaignId'] = 'INVALID'
      else:
        results['campaignId'] = 'VALID'
    
    if countryDesc:
      query = f"SELECT COUNTRY_DESCRIPTION FROM VALIDATION.COUNTRYDATA WHERE COUNTRY_DESCRIPTION = '{countryDesc}' "
      #print("countrydesc" + str(cc.sql(query).collect().empty))
      if cc.sql(query).collect().empty:
        results['countryDesc'] = 'INVALID'
      else:
        results['countryDesc'] = 'VALID'

    if country:
      query = f"SELECT COUNTRY_CODE FROM VALIDATION.COUNTRYDATA WHERE COUNTRY_CODE = '{country}'"
      #print("country" + str(cc.sql(query).collect().empty))
      if cc.sql(query).collect().empty:
        results['country'] = 'INVALID'
      else:
        results['country'] = 'VALID'
    
    if mastercode:
      query = f"SELECT MASTER_CODE FROM VALIDATION.MASTERCODE_DATA WHERE MASTER_CODE = '{mastercode}'"
      #print("mastercode" + str(cc.sql(query).collect().empty))
      if cc.sql(query).collect().empty:
        results['mastercode'] = 'INVALID'
      else:
        results['mastercode'] = 'VALID'
  else:
    return bottle.HTTPResponse(status=400, headers={"content-type": "text/plain"})
  return bottle.HTTPResponse(status=200, body= results, headers={"content-type": "application/json"})